In [1]:
#pip install vegas datasets


In [2]:
import dtale
from vega_datasets import data
import pyforest

In [3]:
data.list_datasets()


['7zip',
 'airports',
 'annual-precip',
 'anscombe',
 'barley',
 'birdstrikes',
 'budget',
 'budgets',
 'burtin',
 'cars',
 'climate',
 'co2-concentration',
 'countries',
 'crimea',
 'disasters',
 'driving',
 'earthquakes',
 'ffox',
 'flare',
 'flare-dependencies',
 'flights-10k',
 'flights-200k',
 'flights-20k',
 'flights-2k',
 'flights-3m',
 'flights-5k',
 'flights-airport',
 'gapminder',
 'gapminder-health-income',
 'gimp',
 'github',
 'graticule',
 'income',
 'iowa-electricity',
 'iris',
 'jobs',
 'la-riots',
 'londonBoroughs',
 'londonCentroids',
 'londonTubeLines',
 'lookup_groups',
 'lookup_people',
 'miserables',
 'monarchs',
 'movies',
 'normal-2d',
 'obesity',
 'ohlc',
 'points',
 'population',
 'population_engineers_hurricanes',
 'seattle-temps',
 'seattle-weather',
 'sf-temps',
 'sp500',
 'stocks',
 'udistrict',
 'unemployment',
 'unemployment-across-industries',
 'uniform-2d',
 'us-10m',
 'us-employment',
 'us-state-capitals',
 'volcano',
 'weather',
 'weball26',
 'wheat',

In [4]:
jdf = data.jobs()

In [5]:
jdf.head()

,job,sex,year,count,perc
0,Accountant / Auditor,men,1850,708,0.000131
1,Accountant / Auditor,men,1860,1805,0.000214
2,Accountant / Auditor,men,1870,1310,0.000100
3,Accountant / Auditor,men,1880,2295,0.000125
4,Accountant / Auditor,men,1900,11753,0.000396


In [6]:
df = dtale.show(data=jdf, ignore_duplicates=True)

In [7]:
cdf = data.cars()

In [8]:
df1 = dtale.show(data=cdf, ignore_duplicates=True)

In [9]:
df1

In [10]:

import pandas as pd

if isinstance(df, (pd.DatetimeIndex, pd.MultiIndex)):
	df = df.to_frame(index=False)

# remove any pre-existing indices for ease of use in the D-Tale code, but this is not required
df = data.cars().reset_index().drop('index', axis=1, errors='ignore')
df.columns = [str(c) for c in df.columns]  # update columns to strings in case they are numbers


In [11]:
df.columns

Index(['Name', 'Miles_per_Gallon', 'Cylinders', 'Displacement', 'Horsepower',
       'Weight_in_lbs', 'Acceleration', 'Year', 'Origin'],
      dtype='object')

In [12]:
df

,Name,Miles_per_Gallon,Cylinders,Displacement,Horsepower,Weight_in_lbs,Acceleration,Year,Origin
0,chevrolet chevelle malibu,18.0,8,307.0,130.0,3504,12.0,1970-01-01,USA
1,buick skylark 320,15.0,8,350.0,165.0,3693,11.5,1970-01-01,USA
2,plymouth satellite,18.0,8,318.0,150.0,3436,11.0,1970-01-01,USA
3,amc rebel sst,16.0,8,304.0,150.0,3433,12.0,1970-01-01,USA
4,ford torino,17.0,8,302.0,140.0,3449,10.5,1970-01-01,USA
...,...,...,...,...,...,...,...,...,...
401,ford mustang gl,27.0,4,140.0,86.0,2790,15.6,1982-01-01,USA
402,vw pickup,44.0,4,97.0,52.0,2130,24.6,1982-01-01,Europe
403,dodge rampage,32.0,4,135.0,84.0,2295,11.6,1982-01-01,USA
404,ford ranger,28.0,4,120.0,79.0,2625,18.6,1982-01-01,USA


In [13]:
import kerastuner as kt
import tensorflow as tf
import tensorflow_datasets as tfds


def build_model(hp):
    """Builds a convolutional model."""
    inputs = tf.keras.Input(shape=(28, 28, 1))
    x = inputs
    for i in range(hp.Int('conv_layers', 1, 3, default=3)):
        x = tf.keras.layers.Conv2D(
            filters=hp.Int('filters_' + str(i), 4, 32, step=4, default=8),
            kernel_size=hp.Int('kernel_size_' + str(i), 3, 5),
            activation='relu',
            padding='same')(x)

        if hp.Choice('pooling' + str(i), ['max', 'avg']) == 'max':
            x = tf.keras.layers.MaxPooling2D()(x)
        else:
            x = tf.keras.layers.AveragePooling2D()(x)

        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.ReLU()(x)

    if hp.Choice('global_pooling', ['max', 'avg']) == 'max':
        x = tf.keras.layers.GlobalMaxPooling2D()(x)
    else:
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)

    optimizer = hp.Choice('optimizer', ['adam', 'sgd'])
    model.compile(optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


def convert_dataset(item):
    """Puts the mnist dataset in the format Keras expects, (features, labels)."""
    image = item['image']
    label = item['label']
    image = tf.dtypes.cast(image, 'float32') / 255.
    return image, label


def main():
    """Runs the hyperparameter search."""
    tuner = kt.Hyperband(
        hypermodel=build_model,
        objective='val_accuracy',
        max_epochs=2,
        factor=2,
        hyperband_iterations=3,
        distribution_strategy=tf.distribute.MirroredStrategy(),
        directory='results_dir',
        project_name='mnist')

    mnist_data = tfds.load('mnist')
    mnist_train, mnist_test = mnist_data['train'], mnist_data['test']
    mnist_train = mnist_train.map(convert_dataset).shuffle(1000).batch(100).repeat()
    mnist_test = mnist_test.map(convert_dataset).batch(100)

    tuner.search(mnist_train,
                 steps_per_epoch=600,
                 validation_data=mnist_test,
                 validation_steps=2,
                 epochs=2,
                 callbacks=[tf.keras.callbacks.EarlyStopping('val_accuracy')])


if __name__ == '__main__':
    main()
#

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.9399999976158142

Best val_accuracy So Far: 0.9900000095367432
Total elapsed time: 00h 03m 57s

Search: Running Trial #6

Hyperparameter    |Value             |Best Value So Far 
conv_layers       |3                 |3                 
filters_0         |12                |16                
kernel_size_0     |5                 |4                 
pooling0          |avg               |max               
filters_1         |32                |12                
kernel_size_1     |3                 |4                 
pooling1          |avg               |max               
filters_2         |8                 |32                
kernel_size_2     |4                 |5                 
pooling2          |max               |max               
global_pooling    |max               |max               
optimizer         |sgd               |adam              
tuner/epochs      |4                 |2                 
tuner/initial_e...|2             

KeyboardInterrupt: 

In [18]:
"""Runs the hyperparameter search."""
tuner = kt.Hyperband(
    hypermodel=build_model,
    objective='val_accuracy',
    max_epochs=2,
    factor=2,
    hyperband_iterations=3,
    distribution_strategy=tf.distribute.MirroredStrategy(),
    directory='results_dir',
    project_name='mnist')
    
tuner.search_space_summary()

2021-04-25 18:32:52,771 - WARNING  - There are non-GPU devices in `tf.distribute.Strategy`, not using nccl allreduce.
2021-04-25 18:32:52,772 - INFO     - Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
2021-04-25 18:32:52,775 - INFO     - Reloading Oracle from existing project results_dir/mnist/oracle.json
2021-04-25 18:32:53,065 - INFO     - Reloading Tuner from results_dir/mnist/tuner0.json


Search space summary
Default search space size: 12
conv_layers (Int)
{'default': 3, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}
filters_0 (Int)
{'default': 8, 'conditions': [], 'min_value': 4, 'max_value': 32, 'step': 4, 'sampling': None}
kernel_size_0 (Int)
{'default': None, 'conditions': [], 'min_value': 3, 'max_value': 5, 'step': 1, 'sampling': None}
pooling0 (Choice)
{'default': 'max', 'conditions': [], 'values': ['max', 'avg'], 'ordered': False}
filters_1 (Int)
{'default': 8, 'conditions': [], 'min_value': 4, 'max_value': 32, 'step': 4, 'sampling': None}
kernel_size_1 (Int)
{'default': None, 'conditions': [], 'min_value': 3, 'max_value': 5, 'step': 1, 'sampling': None}
pooling1 (Choice)
{'default': 'max', 'conditions': [], 'values': ['max', 'avg'], 'ordered': False}
filters_2 (Int)
{'default': 8, 'conditions': [], 'min_value': 4, 'max_value': 32, 'step': 4, 'sampling': None}
kernel_size_2 (Int)
{'default': None, 'conditions': [], 'min_value': 3, 

In [19]:
models = tuner.get_best_models(num_models=2)

In [20]:
tuner.results_summary()

Results summary
Results in results_dir/mnist
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
conv_layers: 3
filters_0: 16
kernel_size_0: 4
pooling0: max
filters_1: 12
kernel_size_1: 4
pooling1: max
filters_2: 32
kernel_size_2: 5
pooling2: max
global_pooling: max
optimizer: adam
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.9900000095367432
Trial summary
Hyperparameters:
conv_layers: 2
filters_0: 8
kernel_size_0: 5
pooling0: avg
filters_1: 28
kernel_size_1: 5
pooling1: max
filters_2: 16
kernel_size_2: 5
pooling2: avg
global_pooling: max
optimizer: adam
tuner/epochs: 8
tuner/initial_epoch: 4
tuner/bracket: 3
tuner/round: 3
tuner/trial_id: 4d42980df8bad138572387b8e5ad1444
Score: 0.9779000282287598
Trial summary
Hyperparameters:
conv_layers: 3
filters_0: 16
kernel_size_0: 4
pooling0: max
filters_1: 12
kernel_size_1: 4
pooling1: max
filters_2: 32
kernel_size_2: 5
pooling2: max
global_pooling: max
optimiz